#### Imports

In [1]:
import pandas as pd
import os
import cProfile
import re
import sys
from pathlib import Path
from datetime import datetime
import numpy as np
from tqdm import tqdm

notebook_dir = Path().resolve()
pyres_dir = str(notebook_dir.parent) + "/PyRes"
sys.path.append(pyres_dir)
import pyres_fof

os.environ["TPTP"] = str(notebook_dir) + "/TPTP-v9.0.0"

### Problem analysis

#### Load Schulz' problem informations

In [2]:
df = pd.read_csv(
    "../notes/protocol_default.csv",
    comment="#", delim_whitespace=True)
df["Usertime"] = df["Usertime"].round(decimals=0)
df = df.sort_values("Usertime", ascending=False)

df

/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_55361/822554389.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


,Problem,Status,Usertime,Failure,Version,Preprocessingtime,Backward_subsumed,Factors_computed,Forward_subsumed,Initial_clauses,Processed_clauses,Resolvents_computed,Systemtime,Tautologies_deleted,Totaltime
13131,SWV194+1.p,T,99.852,success,1.2,-,4,1,1960,357,2166,98975,0.371,2,100.223
5358,LAT261-1.p,T,99.759,success,1.2,-,12,5,5673,3038,3687,61459,0.138,72,99.897
11919,SWB011+1.p,T,99.539,success,1.2,-,25,198,4675,1281,3675,43267,0.120,58,99.659
5094,LAT005-2.p,T,98.176,success,1.2,-,55,0,4391,31,3966,9397,0.032,339,98.208
2021,FLD021-3.p,T,97.506,success,1.2,-,6,8,96,30,1970,277716,0.371,10,97.877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5722,LAT362+3.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5723,LAT362+4.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5724,LAT363+1.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5725,LAT363+2.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-


#### Search all problems in folders

In [4]:
load_from_file = True
if load_from_file:
    df_problems = pd.read_csv("../src/all_problems.csv", index_col=0)
else:
    patterns = {
        "spc": r"(?<=% SPC      :)(\s+\w+)",
        "n_formulae": r"(?<=formulae    :)(\s+\d+)",
        "n_atoms": r"(?<=atoms       :)(\s+\d+)",
    }
    problem_configs = []

    problems_dir = "TPTP-v9.0.0/Problems/"
    dir_list = os.listdir(problems_dir)
    for dir_name in tqdm(dir_list, total=len(dir_list)):
        full_dir_name = os.path.join(problems_dir, dir_name)
        problem_file_list = os.listdir(full_dir_name)
        for filename in problem_file_list:
            full_filename = os.path.join(full_dir_name, filename)
            if not os.path.isfile(full_filename):
                continue
            with open(full_filename) as file:
                text = file.read()
                problem_config: dict = {
                    "Problem": filename,
                }
                for identifier, pattern in patterns.items():
                    re_match = re.search(pattern, text)
                    problem_config[identifier] = re_match.group(0) if re_match is not None else None
                problem_config["full_file_name"] = full_filename
                problem_configs.append(problem_config)

    df_problems = pd.DataFrame.from_records(problem_configs)
df_problems = df_problems.fillna(value=np.nan)
df_problems.to_csv("problems.csv")


df_problems

,Problem,spc,n_formulae,n_atoms,contains_include,full_file_name
0,TOP036+4.p,FOF_THM_RFO_SEQ,34431.0,241872.0,NaN,TPTP-v9.0.0/Problems/TOP/TOP036+4.p
1,TOP029+1.p,FOF_THM_RFO_SEQ,109.0,569.0,NaN,TPTP-v9.0.0/Problems/TOP/TOP029+1.p
2,TOP042+3.p,FOF_THM_RFO_SEQ,13534.0,83921.0,NaN,TPTP-v9.0.0/Problems/TOP/TOP042+3.p
3,TOP008-1.p,CNF_SAT_RFO_NEQ,NaN,NaN,NaN,TPTP-v9.0.0/Problems/TOP/TOP008-1.p
4,TOP009-1.p,CNF_SAT_RFO_NEQ,NaN,NaN,NaN,TPTP-v9.0.0/Problems/TOP/TOP009-1.p
...,...,...,...,...,...,...
26272,MED007+1.p,FOF_THM_RFO_NEQ,41.0,201.0,NaN,TPTP-v9.0.0/Problems/MED/MED007+1.p
26273,MED006+1.p,FOF_THM_RFO_NEQ,41.0,197.0,NaN,TPTP-v9.0.0/Problems/MED/MED006+1.p
26274,MED012+1.p,FOF_SAT_RFO_NEQ,40.0,190.0,NaN,TPTP-v9.0.0/Problems/MED/MED012+1.p
26275,MED008+1.p,FOF_THM_RFO_NEQ,41.0,197.0,NaN,TPTP-v9.0.0/Problems/MED/MED008+1.p


#### Filter for specific problems

In [20]:
df_problem_selection = (
    df_problems
    # [ df_problems["Problem"].str.contains("ITP")
    [ df_problems["Problem"].str.contains("\+")
    & df_problems["spc"].str.contains("NEQ")
    & df_problems["n_formulae"].notna()
    & df_problems["n_atoms"].notna() ]
    .reset_index(drop=True)
)
df_problem_selection.sort_values(by="n_formulae", ascending=False)
# df_problem_selection.describe()

<>:4: SyntaxWarning: invalid escape sequence '\+'
<>:4: SyntaxWarning: invalid escape sequence '\+'
/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_55361/2662562529.py:4: SyntaxWarning: invalid escape sequence '\+'
  [ df_problems["Problem"].str.contains("\+")


,Problem,spc,n_formulae,n_atoms,contains_include,full_file_name
1547,NLP262+1.p,FOF_THM_RFO_NEQ,1026861.0,1026865.0,NaN,TPTP-v9.0.0/Problems/NLP/NLP262+1.p
1559,NLP260+1.p,FOF_THM_RFO_NEQ,1026861.0,1026865.0,NaN,TPTP-v9.0.0/Problems/NLP/NLP260+1.p
1556,NLP261+1.p,FOF_THM_RFO_NEQ,1026861.0,1026865.0,NaN,TPTP-v9.0.0/Problems/NLP/NLP261+1.p
1551,NLP263+1.p,FOF_SAT_RFO_NEQ,1026857.0,1026857.0,NaN,TPTP-v9.0.0/Problems/NLP/NLP263+1.p
1936,MED011+1.p,FOF_SAT_RFO_NEQ,568530.0,568530.0,NaN,TPTP-v9.0.0/Problems/MED/MED011+1.p
...,...,...,...,...,...,...
1025,SYN504+1.p,FOF_THM_EPR_NEQ,1.0,773.0,NaN,TPTP-v9.0.0/Problems/SYN/SYN504+1.p
1024,SYN510+1.p,FOF_THM_EPR_NEQ,1.0,720.0,NaN,TPTP-v9.0.0/Problems/SYN/SYN510+1.p
1023,SYN048+1.p,FOF_THM_EPR_NEQ,1.0,2.0,NaN,TPTP-v9.0.0/Problems/SYN/SYN048+1.p
1021,SYN921+1.p,FOF_THM_EPR_NEQ,1.0,4.0,NaN,TPTP-v9.0.0/Problems/SYN/SYN921+1.p


### Profiling

In [ ]:
opts = [
    ("-s", ""),
    ("-r", "5"),
]
problem_dir = df_problem_selection["full_file_name"][1000]
problem_name = problem_dir[problem_dir.rindex("/")+1:]
args = [
    problem_dir
    # "TPTP-v9.0.0/Problems/AGT/AGT042+1.p"
    # pyres_dir + "/EXAMPLES/PUZ001+1.p"
    # "TPTP-v9.0.0/Problems/MED/MED007+1.p"
    # "TPTP-v9.0.0/Problems/CSR/CSR038+3.p"

]

with cProfile.Profile() as pr:
    pyres_fof.main(from_notebook=True, notebook_opts=opts, notebook_args=args)
    df_function_stats = pd.DataFrame(
        pr.getstats(),
        columns=["func", "ncalls", "ccalls",
                 "tottime", "cumtime", "callers"]
    )
df_function_stats["func"] = df_function_stats["func"].astype(str)
df_function_stats = \
    df_function_stats[ \
        df_function_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]

func_names = []
lines = []
for value in df_function_stats["func"]:
    func_name = re.search(r'(?<=code object )(\w+)', value)
    line = re.search(r'(?<=line )(\d+)', value)
    func_names.append(func_name.group() if func_name is not None else None)
    lines.append(int(line.group()) if line is not None else None)
df_function_stats["func_name"] = func_names
df_function_stats["line"] = lines

df_function_stats = (
        df_function_stats[["func_name", "line", "ncalls", "ccalls",
                  "tottime", "cumtime", "func", "callers"]]
        .sort_values("ncalls", ascending=False)
)


datestring = (
    datetime.now().isoformat()
    .replace("2025-", "")
)
datestring = datestring[:datestring.rindex(".")]

df_function_stats.to_csv(
    str(notebook_dir)
    + f"/profiling/{datestring}_{problem_name}.csv"
    .replace(".p", "")
)
df_function_stats

TPTP-v9.0.0/Problems/SYN/SYN063+1.p
SYN063+1.p
% SZS status Theorem

% Initial clauses    : 31
% Processed clauses  : 42
% Factors computed   : 35
% Resolvents computed: 891
% Tautologies deleted: 0
% Forward subsumed   : 0
% Backward subsumed  : 0
% -------- CPU Time ---------
% User time          : 14.561 s
% System time        : 0.810 s
% Total time         : 15.371 s


/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_55361/2534231915.py:26: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_stats = df_stats[ df_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]


,func_name,line,ncalls,ccalls,tottime,cumtime,func,callers
259,termIsVar,73,355727,0,8.787824e-02,8.776720e-02,"<code object termIsVar at 0x107ac1430, file ""/...",[(<code object _clean_thread_parent_frames at ...
284,termIsCompound,81,240897,0,1.244782e-01,6.480846e-02,"<code object termIsCompound at 0x107ae93e0, fi...","[(<code object __or__ at 0x100a85f10, file ""/o..."
260,termFunc,89,106631,0,8.895831e-02,3.291471e-02,"<code object termFunc at 0x107ac1530, file ""/U...",[(<method '__exit__' of 'sqlite3.Connection' o...
261,termArgs,97,70662,0,6.247266e-02,2.770090e-02,"<code object termArgs at 0x107ac1630, file ""/U...",[(<method 'popleft' of 'collections.deque' obj...
285,isNegative,184,40132,0,4.009462e-03,4.009462e-03,"<code object isNegative at 0x107ae94d0, file ""...",None
...,...,...,...,...,...,...,...,...
195,formulaCNFSplit,640,1,0,6.356660e-04,3.738300e-05,"<code object formulaCNFSplit at 0x107947c10, f...","[(<code object getMatrix at 0x1078d6b80, file ..."
193,tptpLexer,49,1,0,2.320000e-04,5.520900e-05,"<code object tptpLexer at 0x107946870, file ""/...","[(<code object __or__ at 0x100a85f10, file ""/o..."
192,addEqAxioms,160,1,0,1.545420e-04,3.041000e-06,"<code object addEqAxioms at 0x107943850, file ...","[(<code object __init__ at 0x107ae8d50, file ""..."
219,getName,197,1,0,2.500000e-07,2.500000e-07,"<code object getName at 0x107a3e100, file ""/Us...",None


#### Filter for alternating path functions 

In [10]:
df_function_stats = pd.read_csv("profiling/profiling_runs.csv", index_col=0)
df_function_stats[ df_function_stats["func"].str.contains("alternatingpath.py") ]

,func_name,line,ncalls,ccalls,tottime,cumtime,func,callers
235,edge_neighb_of_subset,91,8004594,0,0.729436,0.729436,<code object edge_neighb_of_subset at 0x104c1b...,NaN
280,NaN,98,8004594,0,2.787934,2.058498,"<code object <lambda> at 0x104c79200, file ""/U...",[_lsprof.profiler_subentry(code=<code object e...
279,__init__,19,5006,0,0.000657,0.000657,"<code object __init__ at 0x104c79110, file ""/U...",NaN
278,__init__,9,1702,0,0.000144,0.000144,"<code object __init__ at 0x104c79020, file ""/U...",NaN
272,NaN,87,1702,0,0.000163,0.000163,"<code object <lambda> at 0x104c52b30, file ""/U...",NaN
284,get_neighbours,95,1599,0,4.077462,1.289528,"<code object get_neighbours at 0x104c983b0, fi...",[_lsprof.profiler_subentry(code=<code object <...
270,get_all_nodes,73,1,0,0.000036,0.000036,"<code object get_all_nodes at 0x104c52230, fil...",NaN
163,construct_betweenclause_edges,57,1,0,5.855480,0.215930,<code object construct_betweenclause_edges at ...,[_lsprof.profiler_subentry(code=<code object t...
196,nodes_to_clauses,76,1,0,0.000007,0.000006,"<code object nodes_to_clauses at 0x1043e8030, ...",[_lsprof.profiler_subentry(code=<code object _...
199,get_rel_neighbourhood,109,1,0,4.080163,0.002488,<code object get_rel_neighbourhood at 0x1043f3...,[_lsprof.profiler_subentry(code=<code object c...
